In [ ]:
# ! pip install sounddevice 

In [ ]:
import torch
import sounddevice as sd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import numpy as np
import scipy.signal

# Carrega o modelo e o processador
model_id = "freds0/distil-whisper-large-v3-ptbr"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.eval()

In [ ]:
! pip install faster-whisper


In [2]:
from faster_whisper import WhisperModel

ModuleNotFoundError: No module named 'faster_whisper'